<a href="https://colab.research.google.com/github/JITHIN-ANTONY-JOSEPH/ERP_11358080/blob/main/9_Experiment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Input : Recipe Instructions
### Model : Word2Vec(Text) + Node2Vec(Graph) + Neural Networks(Model) with CROSS VALIDATION

### Mounting to connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Install required libraries

In [ ]:
!pip install node2vec

### Importing required libraries , loading datasets and pre-processing, final_model and evaluation

In [ ]:
import numpy as np
import pandas as pd
import json
import re
import jellyfish
from gensim.models import Word2Vec
from concurrent.futures import ProcessPoolExecutor
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import KFold
import networkx as nx

# Load datasets
flavorgraph_df = pd.read_csv('/content/drive/My Drive/ERP/Dataset/nodes_191120.csv')  # Adjust the path as needed , this is the path to my personal Google Drive
with open('/content/drive/My Drive/ERP/modified_Processed_Layer1.json', 'r') as file:
    recipe1m_data = [json.loads(line) for line in file]
recipe1m_df = pd.DataFrame(recipe1m_data)
substitution_pairs_df = pd.read_csv('/content/drive/My Drive/ERP/Recipe1MSubs_full.csv') # Adjust the path as needed , this is the path to my personal Google Drive

# Merge the datasets based on recipe_id (substitution_pairs_df) and id (recipe1m_df)
merged_df = pd.merge(recipe1m_df, substitution_pairs_df, left_on='id', right_on='recipe_id')

# Example ingredient list for NER-like extraction (replace with your own comprehensive list or use NER model)
ingredient_list = set(flavorgraph_df[flavorgraph_df['node_type'] == 'ingredient']['name'].dropna().unique())

# Function to extract ingredients from instructions
def extract_ingredients_from_instructions(instructions, ingredient_list):
    extracted_ingredients = []
    for instruction in instructions:
        words = instruction.split()
        for word in words:
            if word in ingredient_list:
                extracted_ingredients.append(word)
    return extracted_ingredients

# Apply the extraction function
recipe1m_df['extracted_ingredients'] = recipe1m_df['processed_instructions'].apply(
    lambda instructions: extract_ingredients_from_instructions(instructions, ingredient_list) if isinstance(instructions, list) else []
)

# Prepare sentences for training
sentences = recipe1m_df['extracted_ingredients'].tolist()

# Add substitution contexts to sentences
for _, row in substitution_pairs_df.iterrows():
    ingredient1 = row['ingredient1']
    ingredient2 = row['ingredient2']
    sentences.append([ingredient1, ingredient2])

# Train the Word2Vec model
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=8)  # Increase 'workers' to utilize more CPU cores

# Load the knowledge graph
flavor_graph = nx.read_graphml('/content/drive/My Drive/ERP/knowledge_graph.graphml') # Adjust the path as needed , this is the path to my personal Google Drive

# Filter the graph for ingredient nodes only
def filter_ingredient_nodes(node, attr):
    return node if attr['node_type'] == 'ingredient' else None

ingredient_nodes = [node for node, attr in flavor_graph.nodes(data=True) if filter_ingredient_nodes(node, attr)]
flavor_graph = flavor_graph.subgraph(ingredient_nodes)

# Load the pre-generated Node2Vec model
graph_model = Word2Vec.load('/content/drive/My Drive/ERP/node2vec_model_actual.model') # Adjust the path as needed , this is the path to my personal Google Drive
graph_embeddings = {str(node): graph_model.wv[str(node)] for node in flavor_graph.nodes()}

# Function to combine text and graph embeddings
def get_combined_embedding(ingredient, text_embeddings, graph_embeddings):
    # Check if the ingredient is in the text embeddings
    if ingredient in text_embeddings:
        text_embedding = text_embeddings[ingredient]
    else:
        text_embedding = np.zeros(100)

    # Check if the ingredient is in the graph embeddings
    if ingredient in graph_embeddings:
        graph_embedding = graph_embeddings[ingredient]
    else:
        graph_embedding = np.zeros(100)

    # Combine text and graph embeddings
    combined_embedding = np.concatenate((text_embedding, graph_embedding))
    return combined_embedding

# Define the neural network
class CombinedNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CombinedNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Function to find the top N most similar ingredients based on cosine similarity
def find_top_similar_ingredients(predicted_embedding, combined_embeddings, top_n=10):
    similarities = {}
    for ingredient, embedding in combined_embeddings.items():
        similarity = cosine_similarity(predicted_embedding.reshape(1, -1), embedding.reshape(1, -1))[0][0]
        similarities[ingredient] = similarity
    sorted_ingredients = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
    return [ingredient for ingredient, similarity in sorted_ingredients[:top_n]]

# Function to calculate metrics with Jaro-Winkler similarity threshold
def calculate_metrics(predictions, ground_truths, combined_embeddings, top_n=10, threshold=0.8):
    mrr, hit_1, hit_3, hit_10 = 0.0, 0.0, 0.0, 0.0
    total = len(ground_truths)

    for pred, gt in zip(predictions, ground_truths):
        top_similar = find_top_similar_ingredients(pred, combined_embeddings, top_n=top_n)
        for rank, candidate in enumerate(top_similar, start=1):
            sim = jellyfish.jaro_winkler_similarity(gt, candidate)
            if sim >= threshold:
                mrr += 1.0 / rank
                if rank == 1:
                    hit_1 += 1.0
                if rank <= 3:
                    hit_3 += 1.0
                if rank <= 10:
                    hit_10 += 1.0
                break

    mrr /= total
    hit_1 /= total
    hit_3 /= total
    hit_10 /= total
    return mrr, hit_1, hit_3, hit_10

# Cross-validation setup
kf = KFold(n_splits=5)

# To store results across folds
all_mrr, all_hit_1, all_hit_3, all_hit_10 = [], [], [], []

for fold, (train_index, val_index) in enumerate(kf.split(substitution_pairs_df), 1):
    print(f"\nStarting Fold {fold}...")

    # Split data
    train_df, val_df = substitution_pairs_df.iloc[train_index], substitution_pairs_df.iloc[val_index]

    # (Re-initialize your model here)
    nn_model = CombinedNN(input_dim=200, output_dim=200)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(nn_model.parameters(), lr=0.001)

    # Prepare train data and labels
    train_data = []
    train_labels = []
    for _, row in train_df.iterrows():
        ing1 = row['ingredient1']
        ing2 = row['ingredient2']
        combined_embedding1 = get_combined_embedding(ing1, model.wv, graph_embeddings)
        combined_embedding2 = get_combined_embedding(ing2, model.wv, graph_embeddings)
        train_data.append(combined_embedding1)
        train_labels.append(combined_embedding2)

    train_data = torch.tensor(train_data, dtype=torch.float32)
    train_labels = torch.tensor(train_labels, dtype=torch.float32)

    # Training loop
    for epoch in range(300):
        nn_model.train()
        optimizer.zero_grad()
        outputs = nn_model(train_data)
        loss = criterion(outputs, train_labels)
        loss.backward()
        optimizer.step()

        # Print loss for every epoch
        print(f"Fold {fold} | Epoch {epoch+1} | Loss: {loss.item():.4f}")

    # Prepare validation data for the first 500 entries
    val_data = []
    val_labels = []

    for _, row in val_df.iterrows():
        if len(val_data) >= 500:
            break
        ing1 = row['ingredient1']
        combined_embedding = get_combined_embedding(ing1, model.wv, graph_embeddings)

        val_data.append(combined_embedding)
        val_labels.append(row['ingredient2'])

    val_data = np.array(val_data)
    val_data = torch.tensor(val_data, dtype=torch.float32)
    val_predictions = nn_model(val_data).detach().numpy()

    # Calculate metrics for this fold
    combined_embeddings = {ingredient: get_combined_embedding(ingredient, model.wv, graph_embeddings) for ingredient in model.wv.index_to_key}
    mrr, hit_1, hit_3, hit_10 = calculate_metrics(val_predictions, val_labels, combined_embeddings)

    all_mrr.append(mrr)
    all_hit_1.append(hit_1)
    all_hit_3.append(hit_3)
    all_hit_10.append(hit_10)

# Compute the average metrics across all folds
avg_mrr = np.mean(all_mrr)
avg_hit_1 = np.mean(all_hit_1)
avg_hit_3 = np.mean(all_hit_3)
avg_hit_10 = np.mean(all_hit_10)

print(f"\nFinal Results after Cross-Validation:")
print(f"Average MRR: {avg_mrr:.4f}, Average Hit@1: {avg_hit_1:.4f}, Average Hit@3: {avg_hit_3:.4f}, Average Hit@10: {avg_hit_10:.4f}")



Starting Fold 1...
Fold 1 | Epoch 1 | Loss: 0.8221
Fold 1 | Epoch 2 | Loss: 0.8026
Fold 1 | Epoch 3 | Loss: 0.7873
Fold 1 | Epoch 4 | Loss: 0.7747
Fold 1 | Epoch 5 | Loss: 0.7644
Fold 1 | Epoch 6 | Loss: 0.7558
Fold 1 | Epoch 7 | Loss: 0.7487
Fold 1 | Epoch 8 | Loss: 0.7430
Fold 1 | Epoch 9 | Loss: 0.7375
Fold 1 | Epoch 10 | Loss: 0.7333
Fold 1 | Epoch 11 | Loss: 0.7292
Fold 1 | Epoch 12 | Loss: 0.7256
Fold 1 | Epoch 13 | Loss: 0.7221
Fold 1 | Epoch 14 | Loss: 0.7189
Fold 1 | Epoch 15 | Loss: 0.7160
Fold 1 | Epoch 16 | Loss: 0.7130
Fold 1 | Epoch 17 | Loss: 0.7101
Fold 1 | Epoch 18 | Loss: 0.7079
Fold 1 | Epoch 19 | Loss: 0.7055
Fold 1 | Epoch 20 | Loss: 0.7034
Fold 1 | Epoch 21 | Loss: 0.7011
Fold 1 | Epoch 22 | Loss: 0.6990
Fold 1 | Epoch 23 | Loss: 0.6972
Fold 1 | Epoch 24 | Loss: 0.6952
Fold 1 | Epoch 25 | Loss: 0.6932
Fold 1 | Epoch 26 | Loss: 0.6914
Fold 1 | Epoch 27 | Loss: 0.6897
Fold 1 | Epoch 28 | Loss: 0.6879
Fold 1 | Epoch 29 | Loss: 0.6869
Fold 1 | Epoch 30 | Loss: 0.6857